In [1]:
from agent import agent_executor
from mapper import MapManager

from ipyleaflet import Map, Marker
import json
from odc import ui
from odc.stac import stac_load
from odc import ui
from odc.algo import colorize, to_rgba
import panel as pn
import pystac
from pystac_client.client import Client

import panel as pn
from langchain.callbacks.base import BaseCallbackHandler

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.chat_models import ChatOpenAI

from langchain import OpenAI
from langchain.tools import BaseTool, StructuredTool
from langchain.agents import initialize_agent, AgentType
from typing import Tuple, Optional

from langchain_tools import StacSearchTool
import param
import pandas as pd
import time


In [2]:
from app import ChatWidget

chat = ChatWidget()

In [3]:
class ChatStreamCallbackHandler(BaseCallbackHandler):
    """A basic Call Back Handler that will update the token on the chat widget provided"""

    def __init__(self, chat: "ChatWidget"):
        self.chat = chat

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.chat.token = token

In [4]:
stream_handler = ChatStreamCallbackHandler(chat=chat)
llm = OpenAI(
    temperature=0,
    max_tokens=1000,
    streaming=True,
    callbacks=[stream_handler],
)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
chat._chat = initialize_agent(
    tools=[StacSearchTool()], 
    llm=llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    streaming=False,
    verbose=True,
    # memory = conversational_memory
    )

In [5]:
value = chat._chat.run("Sentinel images over Seattle, WA in April 2023")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "stacsearch",
  "action_input": {
    "bbox": "-122.3, 47.6, -122.2, 47.7",
    "dtime": "2023-04"
  }
}
```

-122.3, 47.6, -122.2, 47.7

Observation: 8 items were found & saved to ./tmp/items.json
Thought: I have the data I need to answer the question
Action:
```
{
  "action": "Final Answer",
  "action_input": "I found 8 Sentinel images over Seattle, WA in April 2023."
}
```

> Finished chain.


In [6]:
value

'I found 8 Sentinel images over Seattle, WA in April 2023.'

In [7]:
value = chat._chat.arun("Sentinel images over Seattle, WA in April 2023")

In [8]:
# await value

In [9]:
await value



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "stacsearch",
  "action_input": {
    "bbox": "-122.3, 47.6, -122.2, 47.7",
    "dtime": "2023-04-01/2023-04-30"
  }
}
```

-122.3, 47.6, -122.2, 47.7
name='stacsearch' description="\n    query the STAC API, using pystac-client, given:\n    - bbox (str), e.g. '-122.3, 47.6, -122.2, 47.7'\n    - dtime (str)\n    " args_schema=None return_direct=False verbose=False callbacks=None callback_manager=None handle_tool_error=False


TypeError: float() argument must be a string or a real number, not 'tuple'